In [1]:
import minimalmodbus
import serial
import time
from datetime import datetime


class MoistureSensor:
    """
    อ่านค่า Raw Moisture และตรวจสอบสถานะ sensor
    """

    def __init__(
        self,
        port: str = '/dev/ttyUSB0',
        slave_id: int = 1,
        baudrate: int = 19200,
        timeout: float = 1,
        register_one_based: int = 30014,
        register_base: int = 30014,
        decimals: int = 0,
        functioncode: int = 4,
    ):
        # สร้าง Instrument
        self.inst = minimalmodbus.Instrument(port, slave_id, mode=minimalmodbus.MODE_RTU)
        self.inst.serial.baudrate = baudrate
        self.inst.serial.bytesize = 8
        self.inst.serial.parity = serial.PARITY_NONE
        self.inst.serial.stopbits = 1
        self.inst.serial.timeout = timeout
        self.inst.clear_buffers_before_each_transaction = True

        # กำหนด register
        self.one_based = register_one_based
        self.base_address = register_base
        self.decimals = decimals
        self.functioncode = functioncode

    def read_raw_moisture(self) -> float:
        """
        อ่านค่า raw moisture และคืนเป็น float
        """
        zero_addr = self.one_based - self.base_address
        return self.inst.read_register(zero_addr, self.decimals, functioncode=self.functioncode)

    def get_status(self) -> dict:
        """
        ตรวจสอบสถานะ sensor:
        - online พร้อมค่า raw moisture
        - no_response ถ้าไม่มีการตอบกลับ
        - error กรณีอื่นๆ
        คืน dict พร้อมสถานะ, ค่า, และ timestamp
        """
        try:
            # value = self.read_raw_moisture()
            return {
                "status": "online",
                "timestamp": datetime.utcnow().isoformat()
            }
        except minimalmodbus.NoResponseError:
            return {
                "status": "no_response",
                "timestamp": datetime.utcnow().isoformat()
            }
        except Exception as e:
            return {
                "status": "error",
                "error": str(e),
                "timestamp": datetime.utcnow().isoformat()
            }


if __name__ == "__main__":
    sensor = MoistureSensor(
        port='/dev/ttyUSB0',
        slave_id=1,
        baudrate=19200,
        timeout=1,
        register_one_based=30014,
        register_base=30014
    )

    try:
        while True:
            raw = sensor.read_raw_moisture()
            print(raw)
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopped by user")



1231
1208
1214
1206
1224
1213
1284
Stopped by user


In [1]:
import time
from datetime import datetime
from app.config import Config
from app.services.sensor_service import MoistureSensor

def read_sensor_batch(
    interval: float = 1.0,
    start_threshold: float = 100,
    end_threshold: float = 100,
    max_below: int = 5
) -> dict:
    """
    รอให้ sensor ตรวจจับวัตถุดิบ (raw moisture > start_threshold) แล้วเก็บค่าต่อเนื่อง
    จนกว่า raw moisture จะต่ำกว่า end_threshold ติดต่อกันเกิน max_below ครั้ง
    คืนเป็น list ของ dict แต่ละรอบจะมี
    - id: timestamp ของการอ่าน (UTC ISO)
    - moisture: ค่า raw moisture
    - temperature: ค่า material temperature
    - status_flags: รายการ flags
    - timestamp: เวลาที่อ่าน
    """
    sensor = MoistureSensor(
        port=Config.MOISTURE_SERIAL_PORT,
        slave_id=Config.MOISTURE_SERIAL_SLAVE_ID,
        baudrate=Config.MOISTURE_SERIAL_BAUDRATE,
        timeout=Config.MOISTURE_SERIAL_TIMEOUT,
        moisture_register=Config.MOISTURE_REGISTER_ONE_BASED,
        status_register=Config.SENSOR_STATUS_REGISTER,
        temp_register=Config.MATERIAL_TEMPERATURE_REGISTER,
        moisture_decimals=Config.MOISTURE_REGISTER_DECIMALS,
        temp_decimals=1,
        functioncode=4,
    )

    readings = []
    started = False
    below_count = 0

    print(f"Waiting for batch start (threshold > {start_threshold}):")
    while True:
        status = sensor.get_status()
        raw = status.get("moisture")
        now = datetime.utcnow().isoformat()
        temp = status.get("temperature")
        flags = status.get("status_flags")
        print(f"[{now}] Moisture={raw}, Temp={temp}, Flags={flags}")

        if not started:
            # เริ่ม batch เมื่อ raw > start_threshold
            if raw is not None and raw > start_threshold:
                started = True
                print(f"Batch started at {now}")
                readings.append({
                    "id": now,
                    "moisture": raw,
                    "temperature": temp,
                    "status_flags": flags,
                    "timestamp": now,
                })
        else:
            # ในระหว่าง batch ใช้ end_threshold ตรวจจบ
            if raw is not None and raw > end_threshold:
                readings.append({
                    "id": now,
                    "moisture": raw,
                    "temperature": temp,
                    "status_flags": flags,
                    "timestamp": now,
                })
                below_count = 0
            else:
                below_count += 1
                print(f"Below end threshold count: {below_count}/{max_below}")
                if below_count >= max_below:
                    print(f"Batch ended at {now} after {below_count} consecutive below-threshold readings")
                    break

        time.sleep(interval)

    print(f"Collected {len(readings)} readings")
    return {"sensor_data": readings}

if __name__ == "__main__":
    # ทดสอบ batch detection, ปรับค่า end_threshold เป็น 200
    data = read_sensor_batch(interval=1.0, start_threshold=1500, end_threshold=1500, max_below=5)
    import json
    print(json.dumps(data, indent=2))



Loaded common env: /home/qcnm2/corn-moisture-inline/.env.common
Loaded sensor env: /home/qcnm2/corn-moisture-inline/services/moisture-sensor/.env.sensor
Waiting for batch start (threshold > 1500):
[2025-05-20T04:42:10.520633] Moisture=194, Temp=19.4, Flags=['Cal Out of Range (Live)', 'Cal Out of Range (Latched)', 'Supply 5V Error (Live)', 'Supply 5V Error (Latched)', 'RF Ref Error (Latched)']
[2025-05-20T04:42:11.745879] Moisture=194, Temp=18.1, Flags=['Cal Out of Range (Live)', 'Cal Out of Range (Latched)', 'Supply 5V Error (Live)', 'Supply 5V Error (Latched)', 'RF Ref Error (Latched)']
[2025-05-20T04:42:12.971491] Moisture=181, Temp=18.6, Flags=['Cal Out of Range (Live)', 'Cal Out of Range (Latched)', 'Supply 5V Error (Live)', 'Supply 5V Error (Latched)', 'RF Ref Error (Latched)']
[2025-05-20T04:42:14.212394] Moisture=182, Temp=17.6, Flags=['Cal Out of Range (Live)', 'Cal Out of Range (Latched)', 'Supply 5V Error (Live)', 'Supply 5V Error (Latched)', 'RF Ref Error (Latched)']
[2025-0